# **Dictionary and Sets - Properties**

Dictionaries and Sets are great to hold data which dont have a inherent order and unique. Another important characteristic is they should be hashable. Due to this hashable property we are able to access/search items in a list with O(1) time complexity. Also their insertion time is O(1)(depends on the hash function).

But those compelling properties comes with a cost like additional memory footprint, additional complexiy/time cost for other operations etc.

Dictionaries and sets work using hash functions and tables. There we first define a memory area and using the Hash function we identify where we need to put the element(index). (Knowledge about hash collisions and all the other craps related to hasp maps will be needed!)


But basically what happens is when we try to insert a value to dictionary, first we calculate the hash and then we mask it. This effectively turn the hash value (very large value) to a index in an array (or put the value in to one of the predefined buckets)

* eg:- 
    - hash value = 28794
    - num of buckets = 8
    - then index = 28794 & 0b111 = 2 so the second index.

The mask value of this case is 0b111 since we only have 8 memoy blocks. But if we have more then we need to make the mask resized as well.

Then we need to check the bucket/index is already in use. If thats the case we can check the current value with existing value. If they are same, then no issue. Else we need to find a new place to add.

> Apparently in python instead of storing values in predefined buckets it saves item index in a array. Then use it to access the actual data in the memory. This is a optimization used by python. Because of this property in python dictionary we can keep track of in which order items were added!

To find a new index in case of hash collision, python uses a linear function `probing`. Here we use another mask using the item's higher order hash values. Psuedo implementation is as followed.

In [14]:
def index_sequence(key, mask=0b111, collision_shift=5):
    hash_val = hash(key) # This returns a integer.
    i = hash_val & mask
    yield i

    # If theres a collision this part will help. (very cleaver IMO!)
    while True:
        hash_val >>= collision_shift
        i = (i * 5 + hash_val + 1) & mask
        yield i

> The while(True) in the code helps to avoid continuous hash collisions for hash values which have same bits for corresponding mask part bits. Also when defining a hash function for data types it is important to design them to reduce hash collisions as much as possible.

We do the exact same thing as above for lookups as well.